In [333]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm, norm

In [334]:
def is_unitary(m):
    return norm(np.eye(len(m)) - m.dot(m.T.conj()))
def is_hermitian(m):
    return norm(m-m.T.conj())

In [335]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])

# comment this out when stuff actually works
bases = ['XXY', 'ZXY', 'YYY', 'IYZ'] # these are all commuting
coefficients = [1]*len(bases)
hamiltonian = np.zeros((2**len(bases[0]), 2**len(bases[0])), dtype=np.cdouble)
for c, lbl in zip(coefficients, bases):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

In [336]:
hamiltonian

array([[0.+0.j, 0.+0.j, 0.-1.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+1.j],
       [0.+0.j, 0.+0.j, 0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+1.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+1.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.-1.j, 0.+0.j, 0.+0.j]])

In [337]:
is_hermitian(hamiltonian)

0.0

In [338]:
def get_circuit(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    for i in range(n):
        for c, b in zip(coefficients, bases):
            # implement e^{-i*c*b*t/n}
            b = b[::-1]
            q = []
            qc.barrier()
            for i, op in enumerate(b):
                if (op == 'X'):
                    qc.h(i)
                    q.append(i)
                elif (op == 'Z'):
                    q.append(i)
                elif (op == 'Y'):
                    qc.sdg(i)
                    qc.h(i)
                    q.append(i)
                elif (op == 'I'):
                    continue
            qc.barrier()
            if (len(q) > 0):
                if (len(q) > 1):
                    for i in range(len(q)-1):
                        qc.cx(q[i], q[-1])
                qc.rz(2*c*t/n, q[-1])
                if (len(q) > 1):
                    for i in reversed(range(len(q)-1)):
                        qc.cx(q[i], q[-1])
                qc.barrier()
                for i, op in enumerate(b):
                    if (op == 'X'):
                        qc.h(i)
                    elif (op == 'Z'):
                        continue
                    elif (op == 'Y'):
                        qc.h(i)
                        qc.s(i)
                    elif (op == 'I'):
                        continue
            qc.barrier()
    return qc

def get_unitaries(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [339]:
get_circuit(n=1).draw()

░ ┌─────┐┌───┐ ░                               ░ ┌───┐┌───┐ ░  ░ ┌─────┐»
q_0: ─░─┤ Sdg ├┤ H ├─░───■───────────────────────■───░─┤ H ├┤ S ├─░──░─┤ Sdg ├»
      ░ └┬───┬┘└───┘ ░   │                       │   ░ ├───┤└───┘ ░  ░ └┬───┬┘»
q_1: ─░──┤ H ├───────░───┼────■─────────────■────┼───░─┤ H ├──────░──░──┤ H ├─»
      ░  ├───┤       ░ ┌─┴─┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐ ░ ├───┤      ░  ░  └───┘ »
q_2: ─░──┤ H ├───────░─┤ X ├┤ X ├┤ Rz(2) ├┤ X ├┤ X ├─░─┤ H ├──────░──░────────»
      ░  └───┘       ░ └───┘└───┘└───────┘└───┘└───┘ ░ └───┘      ░  ░        »
«     ┌───┐ ░                               ░ ┌───┐┌───┐ ░  ░ ┌─────┐┌───┐ ░ »
«q_0: ┤ H ├─░───■───────────────────────■───░─┤ H ├┤ S ├─░──░─┤ Sdg ├┤ H ├─░─»
«     └───┘ ░   │                       │   ░ ├───┤└───┘ ░  ░ ├─────┤├───┤ ░ »
«q_1: ──────░───┼────■─────────────■────┼───░─┤ H ├──────░──░─┤ Sdg ├┤ H ├─░─»
«           ░ ┌─┴─┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐ ░ └───┘      ░  ░ ├─────┤├───┤ ░ »
«q_2: ──────░─┤ X ├┤ X ├┤ Rz(2) ├┤ X ├┤ X ├─░────────────░──░─┤ Sdg ├┤ H ├─░─»
«           ░ └───┘└───┘└───────┘└───┘└───┘ ░            ░  ░ └─────┘└───┘ ░ »
«                                   ░ ┌───┐┌───┐ ░  ░              ░      »
«q_0: ──■───────────────────────■───░─┤ H ├┤ S ├─░──░──────────────░───■──»
«       │                       │   ░ ├───┤├───┤ ░  ░ ┌─────┐┌───┐ ░ ┌─┴─┐»
«q_1: ──┼────■─────────────■────┼───░─┤ H ├┤ S ├─░──░─┤ Sdg ├┤ H ├─░─┤ X ├»
«     ┌─┴─┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐ ░ ├───┤├───┤ ░  ░ └─────┘└───┘ ░ └───┘»
«q_2: ┤ X ├┤ X ├┤ Rz(2) ├┤ X ├┤ X ├─░─┤ H ├┤ S ├─░──░──────────────░──────»
«     └───┘└───┘└───────┘└───┘└───┘ ░ └───┘└───┘ ░  ░              ░      »
«                    ░            ░ 
«q_0: ───────────■───░────────────░─
«     ┌───────┐┌─┴─┐ ░ ┌───┐┌───┐ ░ 
«q_1: ┤ Rz(2) ├┤ X ├─░─┤ H ├┤ S ├─░─
«     └───────┘└───┘ ░ └───┘└───┘ ░ 
«q_2: ───────────────░────────────░─
«                    ░            ░

In [340]:
e_ith, U_circuit = get_unitaries(n=10)

In [341]:
def get_error(e_ith, U_circuit):
    diff = U_circuit.data - e_ith
    # measure the operator's accuracy
    w, v = LA.eig(diff)

    return np.absolute(max(w))

get_error(e_ith, U_circuit)

0.13703550269288342

In [342]:
np.around(e_ith, decimals=3)

array([[ 0.512+0.j,  0.488+0.j, -0.109+0.j, -0.109+0.j, -0.488+0.j,
         0.488+0.j,  0.   +0.j, -0.   +0.j],
       [ 0.488+0.j,  0.512+0.j,  0.109+0.j,  0.109+0.j,  0.488+0.j,
        -0.488+0.j, -0.   +0.j,  0.   +0.j],
       [ 0.109+0.j, -0.109-0.j, -0.464-0.j, -0.488-0.j,  0.   +0.j,
        -0.218-0.j,  0.488+0.j, -0.488-0.j],
       [ 0.109+0.j, -0.109+0.j, -0.488+0.j, -0.464+0.j,  0.218+0.j,
         0.   +0.j, -0.488+0.j,  0.488+0.j],
       [-0.488-0.j,  0.488+0.j,  0.   +0.j, -0.218-0.j, -0.464-0.j,
        -0.488-0.j, -0.109-0.j,  0.109+0.j],
       [ 0.488+0.j, -0.488+0.j,  0.218+0.j,  0.   +0.j, -0.488+0.j,
        -0.464+0.j, -0.109+0.j,  0.109+0.j],
       [-0.   +0.j,  0.   +0.j,  0.488+0.j, -0.488+0.j,  0.109+0.j,
         0.109+0.j,  0.512+0.j,  0.488+0.j],
       [ 0.   +0.j, -0.   +0.j, -0.488+0.j,  0.488+0.j, -0.109+0.j,
        -0.109+0.j,  0.488+0.j,  0.512+0.j]])

In [343]:
np.around(U_circuit, decimals=3)

array([[ 0.51 +0.j,  0.49 +0.j, -0.061-0.j, -0.061-0.j, -0.496-0.j,
         0.496+0.j, -0.   -0.j,  0.   -0.j],
       [ 0.49 +0.j,  0.51 +0.j,  0.061+0.j,  0.061-0.j,  0.496+0.j,
        -0.496-0.j,  0.   +0.j, -0.   +0.j],
       [ 0.158+0.j, -0.158-0.j, -0.461-0.j, -0.48 -0.j, -0.097-0.j,
        -0.219-0.j,  0.474+0.j, -0.474-0.j],
       [ 0.158+0.j, -0.158-0.j, -0.48 -0.j, -0.461-0.j,  0.219+0.j,
         0.097+0.j, -0.474-0.j,  0.474+0.j],
       [-0.474-0.j,  0.474+0.j, -0.097+0.j, -0.219-0.j, -0.461-0.j,
        -0.48 -0.j, -0.158-0.j,  0.158-0.j],
       [ 0.474+0.j, -0.474-0.j,  0.219+0.j,  0.097-0.j, -0.48 -0.j,
        -0.461-0.j, -0.158+0.j,  0.158-0.j],
       [ 0.   +0.j, -0.   -0.j,  0.496+0.j, -0.496-0.j,  0.061+0.j,
         0.061+0.j,  0.51 +0.j,  0.49 +0.j],
       [ 0.   -0.j, -0.   +0.j, -0.496+0.j,  0.496+0.j, -0.061+0.j,
        -0.061+0.j,  0.49 -0.j,  0.51 -0.j]])

In [344]:
hamiltonian

array([[0.+0.j, 0.+0.j, 0.-1.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+1.j],
       [0.+0.j, 0.+0.j, 0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+1.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+1.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.-1.j, 0.+0.j, 0.+0.j]])

In [345]:
e_ith-U_circuit.data

array([[ 2.25983183e-03-4.85722573e-15j, -2.25983183e-03-2.35922393e-15j,
        -4.81651788e-02+7.49400542e-16j, -4.81651788e-02+6.93889390e-16j,
         8.35562773e-03+2.22044605e-15j, -8.35562773e-03-1.73472348e-15j,
         1.62447724e-16+5.82867088e-16j, -2.31330382e-16+2.22044605e-16j],
       [-2.25983183e-03-3.15025783e-15j,  2.25983183e-03-2.80331314e-15j,
         4.81651788e-02-8.60422844e-16j,  4.81651788e-02+2.77555756e-16j,
        -8.35562773e-03-1.81799020e-15j,  8.35562773e-03+1.56819002e-15j,
        -5.76420981e-16+0.00000000e+00j,  6.74551760e-16-3.33066907e-16j],
       [-4.92027821e-02-1.99840144e-15j,  4.92027821e-02+1.38777878e-15j,
        -2.98988694e-03+1.62370117e-15j, -7.50955061e-03+2.16493490e-15j,
         9.73679610e-02+6.38378239e-16j,  1.03760329e-03+3.88578059e-16j,
         1.36053465e-02-9.27335858e-16j, -1.36053465e-02+8.68457391e-16j],
       [-4.92027821e-02-8.60422844e-16j,  4.92027821e-02+2.22044605e-16j,
        -7.50955061e-03+1.38777878e

10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)

# New Stuff

In [346]:
def buildPauliString(qc, pauliString, t, n):
    c, b = pauliString
    # implement e^{-i*c*b*t/n}
    b = b[::-1]
    q = []
    qc.barrier()
    for i, op in enumerate(b):
        if (op == 'X'):
            qc.h(i)
            q.append(i)
        elif (op == 'Z'):
            q.append(i)
        elif (op == 'Y'):
            qc.sdg(i)
            qc.h(i)
            q.append(i)
        elif (op == 'I'):
            continue
    qc.barrier()
    if (len(q) > 0):
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        qc.rz(2*c*t/n, q[-1])
        if (len(q) > 1):
            for i in reversed(range(len(q)-1)):
                qc.cx(q[i], q[-1])
        qc.barrier()
        for i, op in enumerate(b):
            if (op == 'X'):
                qc.h(i)
            elif (op == 'Z'):
                continue
            elif (op == 'Y'):
                qc.h(i)
                qc.s(i)
            elif (op == 'I'):
                continue
    qc.barrier()
    return qc

def buildHamiltonian(qc, pauliStrings, t, n):
    for pauliString in pauliStrings:
        # print(pauliString[1])
        qc = buildPauliString(qc, pauliString, t, n)
    return qc

In [347]:
def testMethods(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n)
        
    e_ith = expm(-1.j * t * hamiltonian)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return get_error(e_ith, U_circuit)

# Higher order trotter formulation.
def get_circuit_2(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    reversedPauliStrings = list(zip(reversed(coefficients), reversed(bases)))
    
    for i in range(n):
        for pauliString in pauliStrings:
            print(f"{pauliString[1]} -> ")
            qc = buildPauliString(qc, pauliString, t, n * 2)
            qc = buildHamiltonian(qc, reversedPauliStrings, t, n * 2)
        # qc = buildHamiltonian(qc, pauliStrings, t, n)
    return qc

# Higher order trotter formulation.
def get_circuit_3(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    reversedPauliStrings = list(zip(reversed(coefficients), reversed(bases)))
    
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n * 2)
        qc = buildHamiltonian(qc, reversedPauliStrings, t, n * 2 / len(pauliStrings))
        # qc = buildHamiltonian(qc, pauliStrings, t, n)
    return qc

# Higher order trotter formulation.
def get_circuit_4(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    reversedPauliStrings = list(zip(reversed(coefficients), reversed(bases)))
    
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n * 2)
        qc = buildHamiltonian(qc, reversedPauliStrings, t, n * 2)
        # qc = buildHamiltonian(qc, pauliStrings, t, n)
    return qc

def get_unitaries_2(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit_2(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

def get_unitaries_3(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit_3(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

def get_unitaries_4(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit_4(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [348]:
testMethods(n=100)

0.013632494586225532

In [353]:
e_ith4, U_circuit4 = get_unitaries_4(n=10)

In [354]:
get_error(e_ith4, U_circuit4)

0.5854223066053028

In [351]:
np.around(e_ith4, decimals=3)

array([[ 0.512+0.j,  0.488+0.j, -0.109+0.j, -0.109+0.j, -0.488+0.j,
         0.488+0.j,  0.   +0.j, -0.   +0.j],
       [ 0.488+0.j,  0.512+0.j,  0.109+0.j,  0.109+0.j,  0.488+0.j,
        -0.488+0.j, -0.   +0.j,  0.   +0.j],
       [ 0.109+0.j, -0.109-0.j, -0.464-0.j, -0.488-0.j,  0.   +0.j,
        -0.218-0.j,  0.488+0.j, -0.488-0.j],
       [ 0.109+0.j, -0.109+0.j, -0.488+0.j, -0.464+0.j,  0.218+0.j,
         0.   +0.j, -0.488+0.j,  0.488+0.j],
       [-0.488-0.j,  0.488+0.j,  0.   +0.j, -0.218-0.j, -0.464-0.j,
        -0.488-0.j, -0.109-0.j,  0.109+0.j],
       [ 0.488+0.j, -0.488+0.j,  0.218+0.j,  0.   +0.j, -0.488+0.j,
        -0.464+0.j, -0.109+0.j,  0.109+0.j],
       [-0.   +0.j,  0.   +0.j,  0.488+0.j, -0.488+0.j,  0.109+0.j,
         0.109+0.j,  0.512+0.j,  0.488+0.j],
       [ 0.   +0.j, -0.   +0.j, -0.488+0.j,  0.488+0.j, -0.109+0.j,
        -0.109+0.j,  0.488+0.j,  0.512+0.j]])

In [352]:
np.around(U_circuit4, decimals=3)

array([[ 0.51 +0.j,  0.49 +0.j, -0.11 -0.j, -0.11 -0.j, -0.488-0.j,
         0.488+0.j, -0.   -0.j,  0.   +0.j],
       [ 0.49 +0.j,  0.51 +0.j,  0.11 +0.j,  0.11 -0.j,  0.488+0.j,
        -0.488-0.j,  0.   +0.j, -0.   +0.j],
       [ 0.11 +0.j, -0.11 -0.j, -0.461-0.j, -0.49 -0.j, -0.   -0.j,
        -0.219-0.j,  0.488+0.j, -0.488-0.j],
       [ 0.11 +0.j, -0.11 +0.j, -0.49 -0.j, -0.461-0.j,  0.219+0.j,
        -0.   +0.j, -0.488-0.j,  0.488+0.j],
       [-0.488-0.j,  0.488+0.j, -0.   +0.j, -0.219-0.j, -0.461-0.j,
        -0.49 -0.j, -0.11 -0.j,  0.11 -0.j],
       [ 0.488+0.j, -0.488-0.j,  0.219+0.j, -0.   -0.j, -0.49 -0.j,
        -0.461-0.j, -0.11 +0.j,  0.11 -0.j],
       [-0.   +0.j, -0.   -0.j,  0.488+0.j, -0.488-0.j,  0.11 +0.j,
         0.11 -0.j,  0.51 +0.j,  0.49 +0.j],
       [ 0.   +0.j, -0.   -0.j, -0.488-0.j,  0.488+0.j, -0.11 +0.j,
        -0.11 +0.j,  0.49 -0.j,  0.51 -0.j]])